<a href="https://colab.research.google.com/github/Anonymous1846/Blood-cancer-subtype-classification-using-DCNN/blob/master/Blood_Cancer_Subtype_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the required dependencies !**

In [3]:
'''
Importing all the necessary dependencies for the Multiclass Classification model !
In the end we have verified the keras installtion, by printing out the keras version.

keras/tensorflow - for developing and training the DL(CNN) model !
os,random - for selection of the images from the directory !
Each Model is stored in its respective directory along with its history file !

Problem Statement: To classifiy cancer cells whether it is CML,ALL,AML, or CLL from the input image, using Convolutional Nueral Network models !

'''
from keras.layers import MaxPooling2D,Conv2D,Dropout,Dense,Flatten,BatchNormalization,MaxPool2D,AveragePooling2D #layers
from keras.applications.imagenet_utils import decode_predictions  
from keras.callbacks import ModelCheckpoint,EarlyStopping 
from keras.preprocessing.image import ImageDataGenerator 
from keras.applications.resnet import ResNet50                        
from keras.preprocessing import image
from keras.models import Sequential       
from sklearn.metrics import roc_auc_score    
from sklearn.metrics import roc_curve    
from sklearn.metrics import auc 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report    
from tensorflow.keras.optimizers import RMSprop,Adam                                                             
import tensorflow as tf
from cv2 import imread
import matplotlib.pyplot as plt    
import numpy as np
import pickle                                                                    
import keras
import os 

# reading the image via  a path 
#causes an error if the keras installtion didn't went well !
print(keras.__version__)

2.9.0


# **Loading the dataset from the google drive**

In [ ]:
'''
This is an optional block of code which can be executed if the 
google drive needs to be mounted on different account
'''

from google.colab import drive
drive.mount('/content/drive',force_remount=False)

# **Image Preprocessing for Test classifiers**

In [ ]:
training_images_preprocessed = ImageDataGenerator(rescale=1./255,
                                                  rotation_range=40,
                                                  width_shift_range=0.2,
                                                  height_shift_range=0.2,
                                                  shear_range=0.2,
                                                  zoom_range=0.2,
                                                  horizontal_flip=True,
                                                  vertical_flip=True,
                                                  fill_mode='nearest')

validate_images_preprocessed = ImageDataGenerator(rescale=1./255)

# **Custom Test CNN Model**

In [7]:
'''
The test CNN model to check whether the given cell is
benign/malignant, if cancerous then check
the subtype of the cancer cell !
'''
test_cnn_model_0=Sequential([
                                    Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)), # relu function is used to negate the negative values from the models, replace them with zeros !
                                    MaxPooling2D(2, 2),
                                    Conv2D(32, (3,3), activation='relu'),# The second convolution
                                    MaxPooling2D(2,2),                                    
                                    Conv2D(64, (3,3), activation='relu'),# The third convolution
                                    MaxPooling2D(2,2),                                   
                                    Conv2D(64, (3,3), activation='relu'), # The fourth convolution
                                    MaxPooling2D(2,2),                                   
                                    Conv2D(64, (3,3), activation='relu'), # The fifth convolution
                                    MaxPooling2D(2,2),                                   
                                    Flatten(),                            # Flatten the  results to feed into a DNN                                    
                                    Dense(512, activation='relu'),        # 512 neuron hidden layer
                                    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('normal') and 0 for the other 'cancer'
                                    Dense(5, activation='softmax') # the output will be the range of -1 and 1(graph will be S shaped !)
                                        ])

test_cnn_model_0.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 149, 149, 16)     0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 35, 35, 64)      

# **Model based on AlexNet**

In [8]:
'''
Implementation of the AlexNet architecture, which is an common architecture of the CNN model
The AlexNet Alrchitecture has been modified since, the original one is capable of classifiying
upto 1000 tensors, and requires more computational power !
'''
alexnet_model =Sequential([
                    Conv2D(96, (11, 11), strides=(4,4), activation='relu', input_shape=(227, 227, 3)),
                    BatchNormalization(),
                    MaxPooling2D((3, 3),strides=(2,2)),
                    Conv2D(256, (5, 5),padding='same', activation='relu'),                    
                    MaxPooling2D((3, 3),strides=(2, 2)),
                    Conv2D(384, (3, 3), padding='same',activation='relu'),
                    Conv2D(384, (3, 3), padding='same',activation='relu'),
                    Conv2D(256, (3, 3), padding='same',activation='relu'),
                    MaxPooling2D((3, 3),strides=(2, 2)),
                    Flatten(),
                    Dense(4096, activation='relu'),# two sense layers with 4096 nuerons each                 
                    Dropout(0.5),
                    Dense(4096, activation='relu'),                
                    Dropout(0.5),
                    Dense(5, activation='softmax')# sigmoid used to binary other wise softmax will be used for probabilitic classification!
                                                  #Padding calculation = input_size + 2 * padding_size-(filter_size-1)
])
print(alexnet_model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization_1 (Batc  (None, 55, 55, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 13, 13, 256)      0         
 g2D)                                                            
                                                      

In [9]:
'''
The transfer learning mechanism applied to ResNet 50, which is a Residual network
Last layers is set to trainable , from 165 onwards !
'''

resnet = ResNet50(
    weights="imagenet", 
    include_top=False,
    input_shape=(224,224,3)
)

for layer in resnet.layers[:165]:
   layer.trainable = False
for layer in resnet.layers[165:]:
   layer.trainable = True

resnet_model = Sequential(
                          [resnet,                         
                          Flatten(),
                          Dropout(0.5),
                          Dense(256,activation='relu'),
                          Dropout(0.5),
                          Dense(5,activation='softmax')
                          ])
print(resnet_model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_5 (Flatten)         (None, 100352)            0         
                                                                 
 dropout_6 (Dropout)         (None, 100352)            0         
                                                                 
 dense_12 (Dense)            (None, 256)               25690368  
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 49,279,365
Trainable params: 30,157,317


In [10]:
'''
Compiling the model using the loss function binary crossentropy and RMSProp !

return : None

params : model object 
'''
def compile_model(model):
  model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(lr=1e-5), #optionally use the Adam with learning rate 0.001 - 1e-5
                metrics=['accuracy'])


In [ ]:
compile_model(resnet_model)

In [ ]:
'''
The below callback list is used to moniter the validation accuracy, and the save the best version of the
CNN model. The paths has been specified for later use !

The Classifier model and the history file are saved in the Binary Classifier Models directory, in the format:

Model_Name_Directory
                | 
                |
                 ---- Model_File.h5
                 ---- Model_History.pickle
                 -----Model_Test_Acc.pickle
'''

model_path    = '/content/drive/MyDrive/Binary Classifier Models /Binary Classifier Model 0/Binary_Classifier_Model.h5'
alexnet_path  = '/content/drive/MyDrive/Binary Classifier Models /AlexNet Classifier/AlexNet_Classifier.h5'
vgg19_path    = '/content/drive/MyDrive/Binary Classifier Models /VGG 19/VGG_19v1.h5' 
vgg16_path    = '/content/drive/MyDrive/Binary Classifier Models /VGG 16/VGG_16.h5'
resnet_path   = '/content/drive/MyDrive/Binary Classifier Models /ResNet/ResNet.h5'